## Note this is using the Analysis_0330 csRNA peaks (which shouldnt matter)

## 1. Load the csRNA merged peaks

## 2. Load the findPeaks peaks

## 3. Merge the two

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = '../parameters/params.yaml'
import yaml
import sys
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

#data_folder = doc['data_folder']
#tissues = doc['tissues'].split(',')
#sys.path.append(doc['pipeline_path'])
ref_fa = doc['GENOME_FA']
anno_gff=doc['ANNOTATION']
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']


import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
#from Homer import *

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


/data2/resources/software/homer/v4.11_10242019/bin//homer


In [2]:
from tss.data import annotation
from numpanpar import parallel_df as pardf

params {'PREFIX': 'ncbi_pice_103', 'ROOT_DIR': '/data/isshamie/TSS/', 'DATA_DIR': '/data/isshamie/TSS/NCBI_PICR_103_Processed/', 'DATA_PROCESSED_DIR': '/data/isshamie/TSS/TSS/data/processed/', 'SUPPLEMENTAL_DIR': '/data/isshamie/TSS/Analysis/supplemental/', 'PIPELINE': '/data/isshamie/TSS/tss', 'GENOME_DIR': '/data/isshamie/genome', 'GENOME_FA': '/data/isshamie/genome/picr.fa', 'GENOME_GFF3': '/data/isshamie/genome/updated_final_sort.gff3', 'GENOME_GTF': '/data/isshamie/genome/updated_final_sort.gtf', 'TISSUES': 'BMDM1hKLA,BMDMwt,Brain,FemaleReproductive,Heart,Intestine,', 'META_FILE': '/data/isshamie/TSS/TSS/parameters/raw_metaSamples.tsv'}
fig_utils from mplh
/data2/resources/software/homer/v4.11_10242019/bin//


In [3]:
os.chdir(doc["results"])
genome="GCF"


### Output files

In [4]:
save_dir = join("stable_rna", genome)
out_csrna_merged = join(save_dir, "merged_stable_peaks.tsv")
out_all_csrna_merged = join(save_dir, "all_merged_stable_peaks.tsv")
out_eTSS_csrna_merged = join(save_dir, "eTSS_merged_stable_peaks.tsv.merge")
out_stable_minus_eTSS = join(save_dir, "stable_minus_eTSS.tsv")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

### input files

In [5]:
anno_gff = '/data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.gff',
indir = "stableRNA/Analysis_03302020/csRNA_peaks_tissues.merged/"
merge_f = join(indir, "csRNA_peaks.tissues.merge")
anno_f = join(indir, "csRNA_peaks.tissues.merge.anno")
eTSS_bed_f = join(genome, "eTSS/TSS.bed")
all_merged_f = join(genome, "merged/samples.merge" )
all_merged_anno_f = join(genome,"merged/samples.merge.anno")

In [6]:
mrna_peak = pd.read_csv(join(genome, "genome", "start_site_mRNA.tsv"), sep="\t", index_col=0)
mrna_peak

,Chr,Start,End,Strand,actual_start,ID,Parent,Dbxref,Name,gbkey,...,model_evidence,product,transcript_id,Note,exception,end_range,partial,start_range,inference,Length
XM_027386529.1,NW_020822366.1,722308,722308,-,722308,rna-XM_027386529.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386529.1",XM_027386529.1,mRNA,...,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,XM_027386529.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386528.1,NW_020822366.1,724610,724610,-,724610,rna-XM_027386528.1,gene-Bmp4,"GeneID:100754303,Genbank:XM_027386528.1",XM_027386528.1,mRNA,...,Supporting evidence includes similarity to: 29...,bone morphogenetic protein 4%2C transcript var...,XM_027386528.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386531.1,NW_020822366.1,1085435,1085435,+,1085435,rna-XM_027386531.1,gene-Cdkn3,"GeneID:100752216,Genbank:XM_027386531.1",XM_027386531.1,mRNA,...,Supporting evidence includes similarity to: 5 ...,cyclin dependent kinase inhibitor 3%2C transcr...,XM_027386531.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386533.1,NW_020822366.1,1085744,1085744,+,1085744,rna-XM_027386533.1,gene-Cdkn3,"GeneID:100752216,Genbank:XM_027386533.1",XM_027386533.1,mRNA,...,Supporting evidence includes similarity to: 1 ...,cyclin dependent kinase inhibitor 3%2C transcr...,XM_027386533.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027386534.1,NW_020822366.1,1114413,1114413,-,1114413,rna-XM_027386534.1,gene-Cnih1,"GeneID:103160710,Genbank:XM_027386534.1",XM_027386534.1,mRNA,...,Supporting evidence includes similarity to: 10...,cornichon family AMPA receptor auxiliary prote...,XM_027386534.1,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XM_027434589.1,NW_020824185.1,3893,3893,+,3893,rna-XM_027434589.1,gene-LOC113839146,"GeneID:113839146,Genbank:XM_027434589.1",XM_027434589.1,mRNA,...,Supporting evidence includes similarity to: 3 ...,mothers against decapentaplegic homolog 1-like,XM_027434589.1,NaN,NaN,NaN,True,".,3893",NaN,1
XM_027434591.1,NW_020824186.1,17914,17914,+,17914,rna-XM_027434591.1,gene-LOC113839147,"GeneID:113839147,Genbank:XM_027434591.1",XM_027434591.1,mRNA,...,Supporting evidence includes similarity to: 6 ...,PRAME family member 12-like%2C transcript vari...,XM_027434591.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027434593.1,NW_020824186.1,29432,29432,+,29432,rna-XM_027434593.1,gene-LOC103158952,"GeneID:103158952,Genbank:XM_027434593.1",XM_027434593.1,mRNA,...,Supporting evidence includes similarity to: 2 ...,PRAME family member 12-like%2C transcript vari...,XM_027434593.1,NaN,NaN,NaN,NaN,NaN,NaN,1
XM_027434595.1,NW_020824188.1,2147,2147,+,2147,rna-XM_027434595.1,gene-LOC113839149,"GeneID:113839149,Genbank:XM_027434595.1",XM_027434595.1,mRNA,...,Supporting evidence includes similarity to: 25...,zinc finger protein 431-like%2C transcript var...,XM_027434595.1,NaN,NaN,NaN,NaN,NaN,NaN,1


In [7]:
#!module load homer_v
!pos2bed.pl {merge_f} > {merge_f}.bed


	Converted 223560 peaks total



In [8]:
merged_stable_files = glob.glob("stableRNA/Analysis_03302020/csRNA_peaks/*.tss.txt")
merged_stable_files

['stableRNA/Analysis_03302020/csRNA_peaks/Brain_START1_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Brain_START2_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Brain_START3_STARTinput1_RNASeq2.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Kidney_START1_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Brain_START2_STARTinput1_RNASeq2.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Spleen_START1_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/FemaleReproductive_START1_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Muscle_START2_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Spleen_START2_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/MiscOrgans_START2_STARTinput1_RNASeq1.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/Brain_START2_STARTinput2_RNASeq2.tss.txt',
 'stableRNA/Analysis_03302020/csRNA_peaks/H

In [9]:
merge_df = pd.read_csv(merge_f, sep="\t", index_col=0)
merge_df

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,csRNA_pipeline/csRNA_peaks_tissues/FemaleReproductive.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Heart.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Muscle.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Spleen.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Lung.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Kidney.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Brain.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Liver.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/MiscOrgans.tss.txt.merge,csRNA_pipeline/csRNA_peaks_tissues/Pancreas.tss.txt.merge
ID,,,,,,,,,,,,,,,,,
0,NW_020822601.1,33967773,33967923,+,40.900002,csRNA_pipeline/csRNA_peaks_tissues/Heart.tss.t...,1,NaN,21852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NW_020822455.1,6758790,6758940,+,30.000000,csRNA_pipeline/csRNA_peaks_tissues/Heart.tss.t...,1,NaN,21853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NW_020822461.1,34406811,34406961,-,36.799999,csRNA_pipeline/csRNA_peaks_tissues/Heart.tss.t...,1,NaN,21854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NW_020822687.1,15499622,15499772,-,39.599998,csRNA_pipeline/csRNA_peaks_tissues/Heart.tss.t...,1,NaN,21855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NW_020822632.1,9912312,9912462,+,36.799999,csRNA_pipeline/csRNA_peaks_tissues/Heart.tss.t...,1,NaN,21857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223555,NW_020822406.1,14607236,14607386,-,18.049999,csRNA_pipeline/csRNA_peaks_tissues/MiscOrgans....,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44239,32282
223556,NW_020822558.1,1002132,1002282,+,10.850000,csRNA_pipeline/csRNA_peaks_tissues/MiscOrgans....,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32227,22615
223557,NW_020822467.1,3885361,3885511,-,22.250000,csRNA_pipeline/csRNA_peaks_tissues/MiscOrgans....,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33834,23909


## Save each stable RNA file, then merge those files. 
Then for our merged peak file we use in beginning, merge with these combined stable rnas and take only the overlaps

In [10]:
samples_stable = dict()
for i in merged_stable_files:
    curr = pd.read_csv(i, sep="\t", index_col=0)
    curr = curr[curr["Stable/Unstable"] == "stable"]
    samples_stable[join(save_dir, "stable_" + os.path.basename(i))] = curr
    curr.to_csv(join(save_dir, "stable_" + os.path.basename(i)), sep='\t')



#cmd = f"mergePeaks {' '.split(glob.glob(join(save_dir, 'stable_*')))}
dist = 'given'
input_files = glob.glob(join(save_dir, 'stable_*'))
cmd = f"mergePeaks -d {dist} -strand {' '.join(input_files)} > {out_csrna_merged}"
print(cmd)
os.system(cmd)

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


mergePeaks -d given -strand stable_rna/GCF/stable_Brain_START1_STARTinput2_RNASeq2.tss.txt stable_rna/GCF/stable_Brain_START1_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Liver_START1_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Pancreas_START1_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Brain_START2_STARTinput2_RNASeq1.tss.txt stable_rna/GCF/stable_Liver_START2_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_MiscOrgans_START2_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Kidney_START1_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Brain_START1_STARTinput2_RNASeq1.tss.txt stable_rna/GCF/stable_Muscle_START2_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Brain_START2_STARTinput1_RNASeq2.tss.txt stable_rna/GCF/stable_Kidney_START2_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Lung_START2_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/stable_Brain_START3_STARTinput1_RNASeq2.tss.txt stable_rna/GCF/stable_Spleen_START1_STARTinput1_RNASeq1.tss.txt stable_rna/GCF/s

0

In [11]:
sns.countplot(pd.read_csv(out_csrna_merged, sep='\t', index_col=0)["Total subpeaks"])

## Merge with eTSS

In [12]:
from tss.utils import Homer

In [13]:
Homer.merge_peaks([eTSS_bed_f, out_csrna_merged], out_eTSS_csrna_merged, dist=151, type_merge='')

mergePeaks -d 151 -venn stable_rna/GCF/eTSS_merged_stable_peaks.tsv.merge_venn -matrix stable_rna/GCF/eTSS_merged_stable_peaks.tsv.merge_mat -strand GCF/eTSS/TSS.bed stable_rna/GCF/merged_stable_peaks.tsv > stable_rna/GCF/eTSS_merged_stable_peaks.tsv.merge


In [14]:
pd.read_csv(out_eTSS_csrna_merged + "_venn",sep='\t')

,GCF/eTSS/TSS.bed,stable_rna/GCF/merged_stable_peaks.tsv,Total,Name
0,NaN,X,46818,stable_rna/GCF/merged_stable_peaks.tsv
1,X,NaN,21350,GCF/eTSS/TSS.bed
2,X,X,19097,GCF/eTSS/TSS.bed|stable_rna/GCF/merged_stable_...


In [15]:
eTSS_csrna_merged = pd.read_csv(out_eTSS_csrna_merged,sep="\t",index_col=0)
eTSS_csrna_merged.head()

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,GCF/eTSS/TSS.bed,stable_rna/GCF/merged_stable_peaks.tsv
ID,,,,,,,,,
0,NW_020822381.1,3321150,3321300,-,12.100000,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822381.1-3321225-2
1,NW_020822410.1,3731831,3731981,+,50.799999,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822410.1-3731906-1
2,NW_020822601.1,67126802,67126952,-,57.299999,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822601.1-67126877-1
3,NW_020822505.1,1552632,1552782,+,11.900000,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822505.1-1552707-1
4,NW_020822616.1,6038910,6039060,+,1627.942871,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822616.1-6038973-14


### Filter for peaks not seen in eTSS but is stable

In [16]:
stable_minus_eTSS = eTSS_csrna_merged[eTSS_csrna_merged["Parent files"] == out_csrna_merged]
stable_minus_eTSS

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,GCF/eTSS/TSS.bed,stable_rna/GCF/merged_stable_peaks.tsv
ID,,,,,,,,,
0,NW_020822381.1,3321150,3321300,-,12.100000,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822381.1-3321225-2
1,NW_020822410.1,3731831,3731981,+,50.799999,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822410.1-3731906-1
2,NW_020822601.1,67126802,67126952,-,57.299999,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822601.1-67126877-1
3,NW_020822505.1,1552632,1552782,+,11.900000,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822505.1-1552707-1
4,NW_020822616.1,6038910,6039060,+,1627.942871,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822616.1-6038973-14
...,...,...,...,...,...,...,...,...,...
46813,NW_020822438.1,15115356,15115506,+,74.199997,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822438.1-15115431-1
46814,NW_020822461.1,33025279,33025429,+,17.799999,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822461.1-33025354-1
46815,NW_020822644.1,132585,132735,-,48.625000,stable_rna/GCF/merged_stable_peaks.tsv,1,NaN,Merged-NW_020822644.1-132643-4


## Merge with our mergePeaks

In [17]:
cmd = f"mergePeaks -d {dist} -strand {all_merged_f} {out_csrna_merged} -venn {join(save_dir, 'merged_stable_peaks.venn')} > {out_all_csrna_merged} "
print(cmd)
os.system(cmd)
#all_merged_f 

### For overlapping TSS 
merged_df = pd.read_csv(out_all_csrna_merged, sep='\t', index_col=0)
merged_df = merged_df[~(merged_df.isnull().any(axis=1))]
merged_df.to_csv(out_all_csrna_merged, sep='\t')

#Homer.merge_peaks(input_files, output_file, dist='given', type_merge=''):

mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn > stable_rna/GCF/all_merged_stable_peaks.tsv 


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
pd.read_csv("stable_rna/GCF/merged_stable_peaks.venn",sep='\t')

,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv,Total,Name
0,NaN,X,6347,stable_rna/GCF/merged_stable_peaks.tsv
1,X,NaN,481087,GCF/merged/samples.merge
2,X,X,58808,GCF/merged/samples.merge|stable_rna/GCF/merged...


# Take the peaks that are not seen in eTSS

### From stable_minus_eTSS,  extract the IDs and take those that overlap with the merged all peaks.

In [19]:
all_csrna_merged = pd.read_csv(out_all_csrna_merged,sep='\t',index_col=0)
all_csrna_merged

,chr,start,end,strand,Stat,Parent files,Total subpeaks,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv
#name (cmd = mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn),,,,,,,,,
Merged-NW_020822429.1-6852555-2,NW_020822429.1,6852464,6852668,-,32.527054,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,2799,Merged-NW_020822429.1-6852544-3
Merged-NW_020822429.1-8601620-2,NW_020822429.1,8601541,8601697,+,27.842501,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,537618,Merged-NW_020822429.1-8601622-1
Merged-NW_020822519.1-2588673-2,NW_020822519.1,2588568,2588831,-,41.549290,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,21856,Merged-NW_020822519.1-2588648-3
Merged-NW_020822407.1-5171143-3,NW_020822407.1,5170887,5171328,+,22.681282,GCF/merged/samples.merge|stable_rna/GCF/merged...,3,1190,"Merged-NW_020822407.1-5171071-2,Merged-NW_0208..."
Merged-NW_020822615.1-22431564-2,NW_020822615.1,22431441,22431672,-,39.881836,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,1192,Merged-NW_020822615.1-22431582-3
...,...,...,...,...,...,...,...,...,...
Merged-NW_020822454.1-15400704-3,NW_020822454.1,15400490,15400911,-,10.926667,GCF/merged/samples.merge|stable_rna/GCF/merged...,3,"13857,323573",Merged-NW_020822454.1-15400711-2
Merged-NW_020822370.1-25440839-4,NW_020822370.1,25440552,25441131,+,123.615433,GCF/merged/samples.merge|stable_rna/GCF/merged...,4,"165447,195267","Merged-NW_020822370.1-25440729-4,Merged-NW_020..."
Merged-NW_020822529.1-58424804-3,NW_020822529.1,58424583,58425084,+,4.830000,GCF/merged/samples.merge|stable_rna/GCF/merged...,3,"81715,532831",Merged-NW_020822529.1-58424805-1


In [20]:
stable_minus_eTSS[out_csrna_merged]

ID
0         Merged-NW_020822381.1-3321225-2
1         Merged-NW_020822410.1-3731906-1
2        Merged-NW_020822601.1-67126877-1
3         Merged-NW_020822505.1-1552707-1
4        Merged-NW_020822616.1-6038973-14
                       ...               
46813    Merged-NW_020822438.1-15115431-1
46814    Merged-NW_020822461.1-33025354-1
46815      Merged-NW_020822644.1-132643-4
46816    Merged-NW_020822439.1-72613114-1
46817    Merged-NW_020822468.1-5143545-10
Name: stable_rna/GCF/merged_stable_peaks.tsv, Length: 46818, dtype: object

In [21]:
all_csrna_merged[all_merged_f].str.contains(',').sum()

232

### Load samples.merge.anno to then extract the stable_minus_eTSS peaks

In [22]:
merge_df = pd.read_csv(all_merged_f,sep='\t',index_col=0)
merge_df.head()

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,MiscOrgans_START1_and_MiscOrgans_startinput1,Liver_START2_and_Liver_startinput1,Brain_GROCap1_and_Brain_GRO1,...,MiscOrgans_START2_and_MiscOrgans_startinput1,Spleen_START1_and_Spleen_startinput1,Lung_START2_and_Lung_startinput1,CHO_START1_and_CHO_startinput1,FemaleReproductive_START1_and_FemaleReproductive_startinput1,Kidney_START2_and_Kidney_startinput1,Kidney_GROCap1_and_Kidney_GRO1,CHO_GROCap1_and_CHO_GRO1,BMDMwt_GROCap1_and_BMDMwt_GRO1,Spleen_START2_and_Spleen_startinput1
ID,,,,,,,,,,,,,,,,,,,,,
0,NW_020822452.1,16590187,16590584,+,16.227333,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,15,NW_020822452.1-1136,NaN,NW_020822452.1-1139,...,NaN,NaN,NaN,NW_020822452.1-450,NaN,NaN,NW_020822452.1-419,NW_020822452.1-145,NaN,NaN
1,NW_020822597.1,23595304,23595489,+,5.740000,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,16,NW_020822597.1-387,NaN,NW_020822597.1-324,...,NW_020822597.1-103,NW_020822597.1-248,NW_020822597.1-10,NaN,NW_020822597.1-101,NaN,NaN,NaN,NaN,NW_020822597.1-126
2,NW_020822680.1,728607,728829,+,7.579579,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,19,NW_020822680.1-92,NaN,NW_020822680.1-114,...,NaN,NaN,NaN,NW_020822680.1-37,NW_020822680.1-56,NaN,NW_020822680.1-67,NW_020822680.1-19,NW_020822680.1-44,NW_020822680.1-72
3,NW_020822602.1,10883074,10883337,+,3.992000,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,20,NW_020822602.1-204,NaN,NaN,...,NW_020822602.1-254,NW_020822602.1-122,NaN,NW_020822602.1-226,NaN,NaN,NaN,NW_020822602.1-303,NW_020822602.1-594,NW_020822602.1-266
4,NW_020822601.1,37427816,37427999,+,129.031738,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,23,NW_020822601.1-18,NaN,NW_020822601.1-1445,...,NW_020822601.1-52,NW_020822601.1-123,NaN,NW_020822601.1-41,NaN,NaN,NW_020822601.1-1949,NW_020822601.1-992,NW_020822601.1-2287,NW_020822601.1-12


In [23]:
all_merge_anno_df = pd.read_csv(all_merged_anno_f,sep='\t',index_col=0)
all_merge_anno_df.head()

,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,Nearest PromoterID,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type
PeakID (cmd=annotatePeaks.pl data/processed/GCF/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.gff),,,,,,,,,,,,,,,,,,
539323,NW_020822460.1,1084630,1084782,+,66162.929688,MiscOrgans_START1_and_MiscOrgans_startinput1|L...,promoter-TSS (ID=exon-XR_003482412.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003482412.1-1;Parent...,-1.0,ID=exon-XR_003482412.1-1;Parent=rna-XR_0034824...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353477,NW_020822618.1,401845,402000,-,21905.132812,MiscOrgans_START1_and_MiscOrgans_startinput1|L...,promoter-TSS (ID=exon-XR_003487470.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003487470.1-1;Parent...,4.0,ID=exon-XR_003487470.1-1;Parent=rna-XR_0034874...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,NW_020822410.1,4827637,4827815,+,20179.468750,MiscOrgans_START1_and_MiscOrgans_startinput1|L...,promoter-TSS (ID=exon-XM_027391402.1-1;Parent=...,-promoter-TSS (ID=exon-XM_027391402.1-1;Parent...,-11.0,ID=exon-XM_027391402.1-1;Parent=rna-XM_0273914...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349134,NW_020822505.1,689618,689773,-,18150.236328,Brain_GROCap1_and_Brain_GRO1|Heart_START1_and_...,promoter-TSS (ID=exon-XR_003483964.1-10;Parent...,-promoter-TSS (ID=exon-XR_003483964.1-10;Paren...,3.0,ID=exon-XR_003484049.1-1;Parent=rna-XR_0034840...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300005,NW_020822602.1,9283364,9283514,+,15297.872070,MiscOrgans_START1_and_MiscOrgans_startinput1|B...,promoter-TSS (ID=exon-XM_027422995.1-6;Parent=...,-promoter-TSS (ID=exon-XM_027422995.1-6;Parent...,0.0,ID=exon-XR_003486731.1-1;Parent=rna-XR_0034867...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
all_merge_anno_df.loc[~(all_merge_anno_df["Annotation"].fillna('').str.contains("exon"))]["Detailed Annotation"].unique()

array(['Intergenic', nan], dtype=object)

### Get the non-coding RNAs

In [25]:
ncrnas = all_merge_anno_df[all_merge_anno_df['Annotation'].fillna('').str.contains("ncRNA")]
#ncrnas_prom = ncrnas[(ncrnas["Annotation"].fillna('').str.contains("promoter"))]
ncrnas

,Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,Nearest PromoterID,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type
PeakID (cmd=annotatePeaks.pl data/processed/GCF/merged/samples.merge /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna -gff /data/isshamie/genome/hamster/ncbi_anno_103/GCF_003668045.1_CriGri-PICR/GCF_003668045.1_CriGri-PICR_genomic.gff),,,,,,,,,,,,,,,,,,
539323,NW_020822460.1,1084630,1084782,+,66162.929688,MiscOrgans_START1_and_MiscOrgans_startinput1|L...,promoter-TSS (ID=exon-XR_003482412.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003482412.1-1;Parent...,-1.0,ID=exon-XR_003482412.1-1;Parent=rna-XR_0034824...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
353477,NW_020822618.1,401845,402000,-,21905.132812,MiscOrgans_START1_and_MiscOrgans_startinput1|L...,promoter-TSS (ID=exon-XR_003487470.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003487470.1-1;Parent...,4.0,ID=exon-XR_003487470.1-1;Parent=rna-XR_0034874...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349134,NW_020822505.1,689618,689773,-,18150.236328,Brain_GROCap1_and_Brain_GRO1|Heart_START1_and_...,promoter-TSS (ID=exon-XR_003483964.1-10;Parent...,-promoter-TSS (ID=exon-XR_003483964.1-10;Paren...,3.0,ID=exon-XR_003484049.1-1;Parent=rna-XR_0034840...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
509442,NW_020822421.1,11363589,11363741,+,11703.328125,MiscOrgans_START1_and_MiscOrgans_startinput1|L...,promoter-TSS (ID=exon-XR_003480724.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003480724.1-1;Parent...,-2.0,ID=exon-XR_003480724.1-1;Parent=rna-XR_0034807...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418707,NW_020822519.1,30535313,30535519,+,6618.855469,Brain_GROCap1_and_Brain_GRO1|Liver_START1_and_...,promoter-TSS (ID=exon-XR_003484581.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003484581.1-1;Parent...,28.0,ID=exon-XR_003484611.1-1;Parent=rna-XR_0034846...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430755,NW_020822439.1,25030902,25031052,+,0.339000,CHO_GROCap1_and_CHO_GRO1,TTS (ID=exon-XR_003481482.1-2;Parent=rna-XR_00...,-TTS (ID=exon-XR_003481482.1-2;Parent=rna-XR_0...,892.0,ID=exon-XR_003481482.1-2;Parent=rna-XR_0034814...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343524,NW_020822658.1,3706656,3706806,-,0.325000,CHO_GROCap2_and_CHO_GRO1,exon (ID=exon-XR_003488114.1-1;Parent=rna-XR_0...,-exon (ID=exon-XR_003488114.1-1;Parent=rna-XR_...,554.0,ID=exon-XR_003488114.1-1;Parent=rna-XR_0034881...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344967,NW_020822419.1,11645255,11645405,+,0.325000,CHO_GROCap2_and_CHO_GRO1,promoter-TSS (ID=exon-XR_003480616.1-1;Parent=...,-promoter-TSS (ID=exon-XR_003480616.1-1;Parent...,-178.0,ID=exon-XR_003480618.1-1;Parent=rna-XR_0034806...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
ncrnas.shape

(12270, 18)

In [27]:
#merged_stable_minus_eTSS = all_csrna_merged[all_csrna_merged[out_csrna_merged].isin(stable_minus_eTSS[out_csrna_merged].values)][all_merged_f]

#### If comma, just take the first peak
#merge_df.loc[merged_stable_minus_eTSS.values.astype(int)]

stable_minus_eTSS_merged_df = all_merge_anno_df.loc[merged_stable_minus_eTSS.apply(lambda x: x.split(",")[0]).values.astype(int)]
stable_minus_eTSS_merged_df

NameError: name 'merged_stable_minus_eTSS' is not defined

In [ ]:
stable_minus_eTSS_merged_df.loc[stable_minus_eTSS_merged_df.index.isin(ncrnas.index)]

In [ ]:
tmp = ncrnas.loc[ncrnas.index.isin(merged_stable_minus_eTSS.apply(lambda x: x.split(",")[0]).values.astype(int))]
tmp[~(tmp["Annotation"].isnull())]

In [ ]:
tmp[(tmp["Annotation"].isnull())]

### Remove the ncRNA peakss

In [ ]:
stable_minus_eTSS_merged_df = stable_minus_eTSS_merged_df.loc[~(stable_minus_eTSS_merged_df.index.isin(ncrnas.index))]

### Get distance to TSS

In [ ]:
stable_minus_eTSS_merged_df = pardf(stable_minus_eTSS_merged_df,
                         annotation.distance_to_landmarks,
                         func_args=(mrna_peak, 'transcript_id', ('gene', ),
                                    False, True),
                         num_processes=24)

In [ ]:
%matplotlib inline
sns.distplot(stable_minus_eTSS_merged_df["Distance to TSS"])

In [ ]:
sns.distplot(np.clip(np.abs(stable_minus_eTSS_merged_df["Distance to TSS"]), None, 10000))

In [ ]:
sns.distplot(np.log10(np.abs(stable_minus_eTSS_merged_df["Distance to TSS"])+1))

In [ ]:
(np.abs(stable_minus_eTSS_merged_df["Distance to TSS"] )<1000).sum()

In [ ]:
stable_minus_eTSS_merged_df[stable_minus_eTSS_merged_df["Nearest gene"] == "Mgat3"]

In [ ]:
sns.distplot(np.log2(stable_minus_eTSS_merged_df["Peak Score"]))

In [ ]:
stable_minus_eTSS_merged_df["Peak Score"].describe()

### Remove Nearest TSS null

In [ ]:
stable_minus_eTSS_merged_df = stable_minus_eTSS_merged_df.dropna(axis=0,subset=["Nearest TSS"])

## Change the name, and Save stable_minus_eTSS_ncRNA

In [ ]:
stable_minus_eTSS_merged_df['rank'] = stable_minus_eTSS_merged_df.groupby(["Nearest gene", "Nearest TSS"])["Peak Score"].rank(ascending=False,
                                                                                                                             method='first')
stable_minus_eTSS_merged_df["Name"] = stable_minus_eTSS_merged_df.apply(lambda x: x["Nearest gene"] + "_"
                                                                      + x["Nearest TSS"] + "_"
                                                                      + str(int(x["rank"])),axis=1)
stable_minus_eTSS_merged_df

In [ ]:
stable_minus_eTSS_merged_df.set_index("Name",drop=False,verify_integrity=True).to_csv(join(save_dir,"stableTSS_minus_eTSS.tsv"),
                                                                                    sep='\t')

### Convert to bed

In [81]:
!pos2bed.pl {out_csrna_merged} > {out_csrna_merged}.bed


	Converted 66022 peaks total



In [83]:
!pos2bed.pl {out_all_csrna_merged} > {out_all_csrna_merged}.bed


	Converted 58808 peaks total



## Take all stable RNA and get distance to nearest gene 
## See which peaks are left out of the final output (from eTSS or gene_df), and add them. 
## Check if Mgat3 is nearest TSS. 
### How many added? Distance? 
### How many have ambigous - distance to nearby other genes too. If this is the case, can directionally help?
### Is direction properly accounted for? 

In [12]:
all_stable_merge = pd.read_csv(out_all_csrna_merged, sep="\t", index_col=0)
all_stable_merge.head()

,chr,start,end,strand,Stat,Parent files,Total subpeaks,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv
#name (cmd = mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn),,,,,,,,,
Merged-NW_020822429.1-6852555-2,NW_020822429.1,6852464,6852668,-,32.527054,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,2799,Merged-NW_020822429.1-6852544-3
Merged-NW_020822429.1-8601620-2,NW_020822429.1,8601541,8601697,+,27.842501,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,537618,Merged-NW_020822429.1-8601622-1
Merged-NW_020822519.1-2588673-2,NW_020822519.1,2588568,2588831,-,41.549290,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,21856,Merged-NW_020822519.1-2588648-3
Merged-NW_020822407.1-5171143-3,NW_020822407.1,5170887,5171328,+,22.681282,GCF/merged/samples.merge|stable_rna/GCF/merged...,3,1190,"Merged-NW_020822407.1-5171071-2,Merged-NW_0208..."
Merged-NW_020822615.1-22431564-2,NW_020822615.1,22431441,22431672,-,39.881836,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,1192,Merged-NW_020822615.1-22431582-3


In [7]:
stable_merge = pd.read_csv(join(save_dir, 'merged_stable_peaks.tsv'), sep='\t', index_col=0)

In [49]:
# from pandarallel import pandarallel
# pandarallel.initialize()

# def get_nearest_gene(peak_sample_indices, samples_stable):
#     curr_p = peak_sample_indices.iloc[7:]
#     curr_p_samp = curr_p[~(curr_p.isnull())]
#     if curr_p_samp[0].count('TSS') > 1:
#         ind = ','.join(curr_p_samp[0].split(',')[:2])
#         if ind not in samples_stable[curr_p_samp.index[0]].index:
#             print("Not here", curr_p_samp)
#             return np.nan
#         else:
#             return samples_stable[curr_p_samp.index[0]].loc[ind]["Closest TSS gene"]
#     return samples_stable[curr_p_samp.index[0]].loc[curr_p_samp[0]]["Closest TSS gene"]
    

# stable_merge["Nearest txn"] = stable_merge.parallel_apply(get_nearest_gene,args=(samples_stable,),axis=1)

# stable_merge["Nearest Gene"] = stable_merge["Nearest txn"].map(pd.read_csv("GCF/genome/start_site_mRNA.tsv",sep="\t",index_col=0)["gene"])

# stable_merge['Nearest Gene'].value_counts()

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 48 workers


## Get the nearest TSS and Gene for each stable RNA peak. 
-Use distance_to_landmarks, but add optionn of nearest_same_direction
-Need to check if the stable can be bidirectional

In [15]:
def capitalize_label(df, labels=("chr", "start", "end", "strand")):
    cols = df.columns
    new_cols = {}
    for i in cols:
        if i in labels:
            new_cols[i] = i[0].upper() + i[1:]
    df = df.rename(new_cols, axis=1)
    return df

In [14]:
#stable_merge = capitalize_label(stable_merge)

NameError: name 'capitalize_label' is not defined

## Merge stable RNA and the samples.merge, and force_same_strand to allow the nearest gene to be the one in the right direction.

In [18]:
# stable_merge_anno = pardf(stable_merge,
#                    annotation.distance_to_landmarks,
#                    func_args=(mrna_peak, 'transcript_id', ('gene', ), False,
#                               True),
#                    num_processes=24)

# stable_merge_anno[stable_merge_anno["Nearest gene"] == "Mgat3"]

In [16]:
all_stable_merge_anno = pardf(capitalize_label(all_stable_merge),
                         annotation.distance_to_landmarks,
                         func_args=(mrna_peak, 'transcript_id', ('gene', ),
                                    False, True),
                         num_processes=24)

all_stable_merge_anno[all_stable_merge_anno["Nearest gene"] == "Mgat3"]

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv,Nearest gene,Nearest TSS,Distance to TSS,isSameStrand
#name (cmd = mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn),,,,,,,,,,,,,
Merged-NW_020822470.1-7464789-2,NW_020822470.1,7464714,7464864,+,16.190001,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,322872,Merged-NW_020822470.1-7464789-1,Mgat3,NM_001244074.1,-36645.0,True
Merged-NW_020822470.1-7464518-2,NW_020822470.1,7464424,7464613,+,7.117500,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,296878,Merged-NW_020822470.1-7464525-6,Mgat3,NM_001244074.1,-36916.0,True
Merged-NW_020822470.1-7510807-2,NW_020822470.1,7510707,7510959,+,19.467571,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,307626,Merged-NW_020822470.1-7510782-2,Mgat3,NM_001244074.1,9399.0,True
Merged-NW_020822470.1-7476017-2,NW_020822470.1,7475931,7476104,+,109.631248,GCF/merged/samples.merge|stable_rna/GCF/merged...,2,114687,Merged-NW_020822470.1-7476017-2,Mgat3,NM_001244074.1,-25417.0,True


In [20]:
all_stable_merge_anno[~all_stable_merge_anno["Parent files"].str.contains("|")]

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv,Nearest gene,Nearest TSS,Distance to TSS,isSameStrand
#name (cmd = mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn),,,,,,,,,,,,,


In [21]:
all_stable_merge_anno[all_stable_merge_anno["GCF/merged/samples.merge"].isnull()]

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv,Nearest gene,Nearest TSS,Distance to TSS,isSameStrand
#name (cmd = mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn),,,,,,,,,,,,,


In [17]:
all_stable_merge_anno[all_stable_merge_anno["Parent files"] == "stable_rna/GCF/merged_stable_peaks.tsv"]

,Chr,Start,End,Strand,Stat,Parent files,Total subpeaks,GCF/merged/samples.merge,stable_rna/GCF/merged_stable_peaks.tsv,Nearest gene,Nearest TSS,Distance to TSS,isSameStrand
#name (cmd = mergePeaks -d given -strand GCF/merged/samples.merge stable_rna/GCF/merged_stable_peaks.tsv -venn stable_rna/GCF/merged_stable_peaks.venn),,,,,,,,,,,,,


In [30]:
capitalize_label(all_stable_merge_anno).to_csv(out_all_csrna_merged.replace(".tsv",".anno.tsv"),sep='\t')

In [31]:
anno_out_all_csrna_merged = out_all_csrna_merged.replace(".tsv",".anno.tsv")

In [32]:
!pos2bed.pl {anno_out_all_csrna_merged} > {anno_out_all_csrna_merged}.bed


	Converted 58812 peaks total

